In [1]:
import ssm
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import Conspecific_Tracking_ta_edit_117 as cs
from scipy import interpolate 
from scipy import misc
from beating import rotation
from beating import tail_spline

In [2]:
path = "/Users/miaoyuqi/研究/Statistical analyses of Siamese fighting fish aggressive behavior/DSI-Students/Yuqi_scripts/top example.h5"
f = pd.HDFStore(path,'r')
data_top = f.get('df_with_missing')

In [3]:
data_top.columns = data_top.columns.droplevel()
data_top1 = cs.auto_scoring_tracefilter(data_top)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
data_top2 = data_top1[data_top1.isnull()["F_spine1"]["x"]== False].fillna(method = "ffill")

In [5]:
rotate_spline = rotation(data_top2.iloc[90000:95000,:])

In [6]:
y_der = tail_spline(rotate_spline)

/opt/anaconda3/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[0][7]=x[0][0]
  (i, m, i)))
/opt/anaconda3/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[1][7]=x[1][0]
  (i, m, i)))


In [7]:
y_der

[[array([-5.10691697e+01,  2.19869347e+06,  6.62058370e+06,  1.71354645e+07,
          2.80284568e+07,  3.74031518e+07,  7.16691327e+07]),
  array([-2.27901935e+01,  4.13927937e+04,  1.30883290e+05,  3.47080593e+05,
          5.72601770e+05,  7.67258921e+05,  1.48092363e+06])],
 [array([-5.04189009e+01,  2.36954741e+06,  7.90289379e+06,  1.71556131e+07,
          2.72236844e+07,  4.20175859e+07,  6.31491934e+07]),
  array([-4.38360156e+01,  7.22955793e+05,  2.42550846e+06,  5.27740374e+06,
          8.38291462e+06,  1.29480727e+07,  1.94710601e+07])],
 [array([-6.93094762e+01,  1.60985535e+06,  1.15547130e+07,  2.30257488e+07,
          3.66239835e+07,  6.14192520e+07,  1.05068567e+08]),
  array([-2.12710873e+01,  6.94764446e+04,  5.08327710e+05,  1.01631344e+06,
          1.61918524e+06,  2.71931683e+06,  4.65720807e+06])],
 [array([-9.33992794e+01,  2.23661888e+06,  1.22823578e+07,  3.90230519e+07,
          5.80228041e+07,  1.31854234e+08,  1.54149431e+08]),
  array([-1.94656066e+01